<a href="https://colab.research.google.com/github/PondKann/KKU_data_mining/blob/master/Data_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Function

In [21]:
import pandas as pd
import os 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from google.colab import drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
path = '/content/drive/My Drive/DataViz_2021'
os.listdir(path)

['pm-18-jan-2021.csv',
 'covid-19-daily_data_dictionary.xlsx',
 'income.csv',
 'income2.xls',
 'จัดซื้อ 57-58.xls',
 'online_retail_II.xlsx',
 'online_retail_II.csv']

1. นำเข้าข้อมูล และ Package ที่ต้องการ

In [24]:
data_path = os.path.join(path,'online_retail_II.csv')
datacsv = pd.read_csv(os.path.join(path,'online_retail_II.csv'),encoding='utf-8')
df = datacsv

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [26]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


2. ทำความสะอาดข้อมูล โดยการลบรายการที่ถูกยกเลิกออก

In [27]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['Invoice'], inplace=True)
df['Invoice'] = df['Invoice'].astype('str')
df = df[~df['Invoice'].str.contains('C')]

3. Group by ข้อมูล Invoice และ Description โดยให้แสดงจำนวนสินค้าที่ถูกสั่งในแต่ละรายการ (Invoice) ถ้าไม่ได้สั่งสินค้านั้นก็ให้ใส่เลขศูนย์

In [28]:
basket0 = (df.groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

4. แปลงค่าข้อมูลในตาราง basket0 ถ้าค่าข้อมูลเป็น <= 0 ให้เปลี่ยนเป็นเลข 0 แต่ถ้าค่าข้อมูลเป็น >= 1 ให้เปลี่ยนเป็นเลข 1 และลบ รหัสไปรษณีย์ (POSTAGE)

In [29]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket0_sets = basket0.applymap(encode_units)
basket0_sets.drop('POSTAGE', inplace=True, axis=1)

5. กฎความสัมพันธ์ โดยกำหนด confidence >= 0.8

In [32]:
frequent_itemsets0 = apriori(basket0_sets, min_support=0.018, use_colnames=True)
rules0 = association_rules(frequent_itemsets0, metric="lift", min_threshold=1)

AssoAll = rules0[ (rules0['lift'] >= 4)& (rules0['confidence'] >= 0.8)]

In [33]:
AssoAll

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
35,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760
209,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.030713,0.051722,0.026298,0.856240,16.554511,0.024709,6.596260
210,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.029064,0.049248,0.026298,0.904841,18.373184,0.024867,9.991237
215,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.024697,0.051722,0.019796,0.801572,15.497554,0.018519,4.778943
221,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...",(JUMBO BAG RED RETROSPOT),0.022707,0.101504,0.018341,0.807692,7.957236,0.016036,4.672179
227,"(JUMBO STORAGE BAG SUKI, JUMBO BAG PINK POLKADOT)",(JUMBO BAG RED RETROSPOT),0.024988,0.101504,0.020039,0.801942,7.900583,0.017502,4.536523


# Data1

In [31]:
data1 = pd.read_excel(data_path,sheet_name='Year 2009-2010')

XLRDError: ignored

In [ ]:
data1.head()

In [ ]:
data1.count()

## Clean Data

In [ ]:
df1 = data1.dropna()

In [ ]:
df1.dropna().count()

In [ ]:
size_before_drop = 525461
size_after_drop = 417534
print(f'size before drop = {size_before_drop} \nsize after drop = {size_after_drop}\nwhich is {100*((size_before_drop-size_after_drop)/size_before_drop)}')

In [ ]:
df1.head( )

In [ ]:
df11 = df1[['Invoice','StockCode']]
df11

In [ ]:
df1

In [ ]:
type({df1})

In [ ]:
table1 = df1.groupby('Invoice')['StockCode'].apply(list)
table1

In [ ]:
table1 = str('Invoice')
table1

In [ ]:
!pip install apyori
from apyori import apriori

In [ ]:
list(apriori(table1, min_support = 0.02 , min_confidence = 0.01 ))

# Data2

In [ ]:
data2 = pd.read_excel(data_path,sheet_name='Year 2010-2011')

In [ ]:
data2.head()

In [ ]:
data2.count()

In [ ]:
data2.dropna().count()

In [ ]:
size_before_drop = 541910
size_after_drop = 406830
print(f'size before drop = {size_before_drop} \nsize after drop = {size_after_drop}\nwhich is {100*((size_before_drop-size_after_drop)/size_before_drop)}')

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
all_table = pd.concat([data1,data2])
all_table.head()

In [ ]:
!pip install apyori
from apyori import apriori

In [ ]:
table = all_table.groupby('Invoice')['StockCode'].apply(list)
table

In [ ]:
list(apriori(table, min_support = 0.1 , min_confidence = 0.8 ))